In [1]:
from os import listdir
from os.path import isfile, join
# from shapely.geometry import *
# from shapely.wkt import loads
# from shapely import LineString, MultiPoint
# from shapely.ops import nearest_points, split, snap, unary_union

import contextily as cx
import earthaccess
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import os
import pandas as pd
import xarray as xr
import shapely

/work/pi_cjgleason_umass_edu/.conda/envs/narrowPIXC/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get tiles for test val set

#### Finding tiles with geolocation_qual < 4096 (CAN SKIP)

##### Download (one-time)

In [3]:
# Yukon River with Porcupine, Chandalar, etc.
# Ohio River with Kanawha
granules = ['*_358_035L_*','*_147_223R_*']

In [4]:
for granule in granules:
    pixc_results = earthaccess.search_data(short_name = 'SWOT_L2_HR_PIXC_2.0',
                                           # According to Planet, ice-off on Yukon ~May 15, 2024
                                           temporal = ('2024-05-15 00:00:00', '2024-09-03 23:59:59'),
                                           granule_name = granule)
    pixc_results
    # download
# earthaccess.download(pixc_results, '../../SWOTdata/PIXC_v2_0_090324/yukon_ohio_test_tiles_nc/')

In [5]:
pixc_results

[Collection: {'Version': '2.0', 'ShortName': 'SWOT_L2_HR_PIXC_2.0'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -82.59126030069007, 'SouthBoundingCoordinate': 38.55884766869757, 'EastBoundingCoordinate': -81.66195239994386, 'NorthBoundingCoordinate': 39.31755183480832}]}, 'Track': {'Cycle': 16, 'Passes': [{'Pass': 147, 'Tiles': ['223R']}]}}}
 Temporal coverage: {'RangeDateTime': {'EndingDateTime': '2024-06-03T10:38:32.205Z', 'BeginningDateTime': '2024-06-03T10:38:21.117Z'}}
 Size(MB): 342.7403316497803
 Data: ['https://archive.swot.podaac.earthdata.nasa.gov/podaac-swot-ops-cumulus-protected/SWOT_L2_HR_PIXC_2.0/SWOT_L2_HR_PIXC_016_147_223R_20240603T103821_20240603T103832_PIC0_01.nc'],
 Collection: {'Version': '2.0', 'ShortName': 'SWOT_L2_HR_PIXC_2.0'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Latitude': 38.55894266078424, 'Longitude': -82.5914085575929}, {'Latit

**#########
Need to automate only downloading the latest version of each granule**
    For now, deleting earlier versions (e.g. 02 when 03 is present) and
    original when reprocessed is present:
            "Where PIC0 and PGC0 products both exist for a particular time,
            users are advised to use the PGC0 products. This is expected
            when reprocessed LR products are generated for cycles 7-9."

##### Read in and look at flags NOT USING HERE

In [ ]:
# Read in xarray
ds_PIXC = xr.open_mfdataset(paths=leaf_off[4], group = 'pixel_cloud', engine='h5netcdf')

In [ ]:
ds_PIXC

In [ ]:
counts = pd.DataFrame(ds_PIXC.geolocation_qual).value_counts().sort_index()
counts

In [ ]:
counts.sum()

### Check that tiles are what we want

In [6]:
# Paths to test tiles
yukon = '/nas/cee-water/cjgleason/SWOTdata/PIXC_v2_0_090324/yukon_ohio_test_tiles_nc/SWOT_L2_HR_PIXC_018_358_035L_20240722T163225_20240722T163236_PIC0_01.nc'
ohio = '/nas/cee-water/cjgleason/SWOTdata/PIXC_v2_0_090324/yukon_ohio_test_tiles_nc/SWOT_L2_HR_PIXC_020_147_223R_20240825T213837_20240825T213848_PIC0_01.nc'

In [3]:
# Load Yukon global vars
ds_y_glob = xr.open_mfdataset(paths=yukon, engine='h5netcdf')

In [13]:
# ds_y_glob

In [7]:
# load Yukon pixel cloud
ds_y = xr.open_mfdataset(paths=yukon, group='pixel_cloud', engine='h5netcdf')
ds_y

<xarray.Dataset> Size: 1GB
Dimensions:                                (points: 4891635, complex_depth: 2,
                                            num_pixc_lines: 3253)
Coordinates:
    latitude                               (points) float64 39MB dask.array<chunksize=(489164,), meta=np.ndarray>
    longitude                              (points) float64 39MB dask.array<chunksize=(489164,), meta=np.ndarray>
Dimensions without coordinates: points, complex_depth, num_pixc_lines
Data variables: (12/61)
    azimuth_index                          (points) float64 39MB dask.array<chunksize=(978327,), meta=np.ndarray>
    range_index                            (points) float64 39MB dask.array<chunksize=(978327,), meta=np.ndarray>
    interferogram                          (points, complex_depth) float32 39MB dask.array<chunksize=(1222909, 1), meta=np.ndarray>
    power_plus_y                           (points) float32 20MB dask.array<chunksize=(978327,), meta=np.ndarray>
    power_minus_y                          (points) float32 20MB dask.array<chunksize=(978327,), meta=np.ndarray>
    coherent_power                         (points) float32 20MB dask.array<chunksize=(978327,), meta=np.ndarray>
    ...                                     ...
    pixc_line_qual                         (num_pixc_lines) float64 26kB dask.array<chunksize=(3253,), meta=np.ndarray>
    pixc_line_to_tvp                       (num_pixc_lines) float32 13kB dask.array<chunksize=(3253,), meta=np.ndarray>
    data_window_first_valid                (num_pixc_lines) float64 26kB dask.array<chunksize=(3253,), meta=np.ndarray>
    data_window_last_valid                 (num_pixc_lines) float64 26kB dask.array<chunksize=(3253,), meta=np.ndarray>
    data_window_first_cross_track          (num_pixc_lines) float32 13kB dask.array<chunksize=(3253,), meta=np.ndarray>
    data_window_last_cross_track           (num_pixc_lines) float32 13kB dask.array<chunksize=(3253,), meta=np.ndarray>
Attributes:
    description:                 cloud of geolocated interferogram pixels
    interferogram_size_azimuth:  3253
    interferogram_size_range:    4534
    looks_to_efflooks:           1.5377332880566343
    num_azimuth_looks:           7.0
    azimuth_offset:              5

In [ ]:
plt.scatter(x=ds_y.longitude, y=ds_y.latitude, c=ds_y.classification, cmap='Accent')
# plt.clim((0,100))
# plt.colorbar().set_label('Height (m)')

**Ohio**

In [24]:
## Load Ohio global vars
# ds_o_glob = xr.open_mfdataset(paths=ohio, engine='h5netcdf')

In [25]:
# ds_o_glob

In [26]:
## load Ohio tile
# ds_o = xr.open_mfdataset(paths=ohio, group='pixel_cloud', engine='h5netcdf')

In [27]:
# ds_o

### Make unfiltered geopckgs

In [49]:
def makeGDFnon(ds, data_var):
    # This function takes the pixel cloud, makes a GeoDataFrame,
    # renames columns as needed, projects the CRS, and returns
    # the GDF 

    # Create GDF
    gdf_PIXC = gpd.GeoDataFrame(data=ds[data_var],
                                geometry=gpd.points_from_xy(
                                    ds.longitude,
                                    ds.latitude),
                                crs="EPSG:4326") # PIXC has no native CRS, setting same as River_SP

    if data_var == 'classification':
        gdf_PIXC.rename(columns={gdf_PIXC.columns[0]: 'klass'}, inplace=True)
    else:
        gdf_PIXC.rename(columns={gdf_PIXC.columns[0]: data_var}, inplace=True)
    
    # Convert the crs to WGS 84 / Pseudo-Mercator
    gdf_PIXC = gdf_PIXC.to_crs(epsg=3857)
    return gdf_PIXC

In [27]:
# Set up
data_path = '/nas/cee-water/cjgleason/SWOTdata/PIXC_v2_0_090324/yukon_ohio_test_tiles_nc/'
save_path = '/nas/cee-water/cjgleason/SWOTdata/PIXC_v2_0_090324/yukon_ohio_test_tiles_gpkg/'

In [30]:
# Get all granules
files = listdir(join(data_path))

# Remove hidden folders from list
if '.ipynb_checkpoints' in files:
    files.remove('.ipynb_checkpoints')
# files

In [52]:
for file in files:
    ds_PIXC = xr.open_mfdataset(paths=join(data_path, file), group='pixel_cloud', engine='h5netcdf')
    # data_vars = list(ds_PIXC.data_vars)

    # Make PIXC, specify data variable
    gdf_PIXC = makeGDFnon(ds=ds_PIXC, data_var='classification')

    # Write out file
    gdf_PIXC.to_file(os.path.join(save_path, file + '_klass.gpkg'), driver='GPKG')

### Make filtered geopckgs

In [2]:
def bitwiseMask(ds):
    # Fow now, eliminate the really bad stuff
    mask = np.where(np.logical_and(ds.classification > 1, ds.geolocation_qual < 2**12))[0]
    print(mask.shape)
    return mask

In [54]:
def makeGDF(ds, mask, data_var):
    # This function takes the pixel cloud, makes a GeoDataFrame,
    # renames columns as needed, projects the CRS, and returns
    # the GDF 

    # Create GDF
    gdf_PIXC = gpd.GeoDataFrame(ds[data_var][mask],
                                geometry=gpd.points_from_xy(
                                    ds.longitude[mask],
                                    ds.latitude[mask]),
                                crs="EPSG:4326") # PIXC has no native CRS, setting same as River_SP

    if data_var == 'classification':
        gdf_PIXC.rename(columns={gdf_PIXC.columns[0]: 'klass'}, inplace=True)
    else:
        gdf_PIXC.rename(columns={gdf_PIXC.columns[0]: data_var}, inplace=True)
    
    # Convert the crs to WGS 84 / Pseudo-Mercator
    gdf_PIXC = gdf_PIXC.to_crs(epsg=3857)
    return gdf_PIXC

In [55]:
# Set up
data_path = '/nas/cee-water/cjgleason/SWOTdata/PIXC_v2_0_090324/yukon_ohio_test_tiles_nc/'
save_path = '/nas/cee-water/cjgleason/SWOTdata/PIXC_v2_0_090324/yukon_ohio_test_tiles_gpkg/'

In [63]:
# Get all granules
files = listdir(join(data_path))

# Remove hidden folders from list
if '.ipynb_checkpoints' in files:
    files.remove('.ipynb_checkpoints')

# files

In [58]:
for file in files:
    ds_PIXC = xr.open_mfdataset(paths=join(data_path, file), group='pixel_cloud', engine='h5netcdf')
    # data_vars = list(ds_PIXC.data_vars)

    # Make mask
    mask = bitwiseMask(ds=ds_PIXC)

    # If dataframe not empty after filtering
    if mask.shape != (0,):
        # Make PIXC, specify data variable
        gdf_PIXC = makeGDF(ds=ds_PIXC, mask=mask, data_var='classification')

        # Write out file
        gdf_PIXC.to_file(os.path.join(save_path, file + '_filtered_klass.gpkg'), driver='GPKG')

(447918,)
(25025,)
(0,)
(1973689,)
(1483466,)
(2283449,)
